In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
from dpo_utils import *
from dpo_data_module import CombinedForgetRetainDataset
from collators import dpo_retain_collator
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from accelerate import  Accelerator
from config import Config
import torch
from peft import  LoraConfig, get_peft_model
from utils import find_all_linear_names
import pandas as pd
from torch.utils.data import Subset

In [3]:
cfg = Config()

accelerator = Accelerator()

In [5]:
cfg.save_dir = 'outputs/wpu_batch_dpo_2_6'
cfg.save_dir

'outputs/wpu_batch_dpo_2_6'

In [6]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct', token = cfg.access_token)
if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
if tokenizer.pad_token_id is None:
        tokenizer.pad_token_id = tokenizer.eos_token_id

In [7]:
policy_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_id,
    torch_dtype=torch.bfloat16, 
    token=cfg.access_token 
    )
print("Base model loaded.")


# --- Apply LoRA on policy model ---
print("Applying LoRA...")
lora_config = LoraConfig(
    r=cfg.LoRA_r,
    lora_alpha=cfg.LoRA_alpha,
    lora_dropout=cfg.LoRA_dropout,
    target_modules=find_all_linear_names(policy_model), 
    bias='none',
    task_type='CAUSAL_LM',
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Base model loaded.
Applying LoRA...


In [8]:
# Get PEFT model 
model = get_peft_model(policy_model, lora_config)
print("PEFT model created.")
model.print_trainable_parameters()
model.config.use_cache = False # Important for gradient checkpointing


PEFT model created.
trainable params: 20,971,520 || all params: 8,051,232,768 || trainable%: 0.2605


In [9]:
ref_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_id,
    torch_dtype=torch.bfloat16,
    token=cfg.access_token
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
forget = pd.read_csv(cfg.forget_path)
retain = pd.read_csv(cfg.retain_path)

In [11]:
forget['factor'] = -1.0
retain['factor'] = 1.0
forget['factor'] = forget['factor'].astype('float')
retain['factor'] = retain['factor'].astype('float')
retain['idk'] = 'idk'


In [12]:
total_batch_size = 8
n_forget_in_batch = 2
n_retain_in_batch = total_batch_size - n_forget_in_batch
print(f"Batch size: {total_batch_size}, Forget samples in batch: {n_forget_in_batch}, Retain samples in batch: {n_retain_in_batch}")

Batch size: 8, Forget samples in batch: 2, Retain samples in batch: 6


In [13]:
train_dataset =  CombinedForgetRetainDataset(
    forget_df = forget,
    retain_df = retain,
    tokenizer = tokenizer,
    max_length = 256,
    block_size = total_batch_size,
    n_forget   = n_forget_in_batch,
    n_retain   = n_retain_in_batch
)


Combined dataset initialized with 2432 samples.
Verifying sample structure (first few blocks):
  Block 0: 2 forget, 6 retain samples. Expected: 2, 6
  Block 1: 2 forget, 6 retain samples. Expected: 2, 6
  Block 2: 2 forget, 6 retain samples. Expected: 2, 6


In [21]:
training_args = TrainingArguments(
        output_dir = f'{cfg.save_dir}',
        overwrite_output_dir= True,
        max_grad_norm=1.0,
        learning_rate = cfg.lr,
        per_device_train_batch_size= 1, 
        num_train_epochs= cfg.num_epochs,
        weight_decay = cfg.weight_decay,
        logging_dir = f'{cfg.save_dir}/logs',
        logging_steps= 250,
        eval_strategy= 'no',
        label_names = ['labels'],
        bf16 = True,
        gradient_accumulation_steps= 8,
        remove_unused_columns=False,
        report_to = 'wandb',
        seed = 42,
)

In [22]:
# trainer = BatchRetainNPOTrainer(
#      model = model,
#      ref_model= ref_model,
#      args = training_args,
#      train_dataset = train_dataset, 
#      data_collator = dpo_retain_collator,
#      beta=cfg.npo_beta,
#)

trainer = BatchRetainDPOTrainer(
    model = model,
    ref_model= ref_model,
    args = training_args,
    train_dataset = train_dataset, 
    data_collator = dpo_retain_collator,
    beta=cfg.npo_beta,
)

Preparing reference model...
Rank 0: In _prepare_ref_model. Accelerator device: cuda, torch.cuda.current_device(): 0
Reference model prepared and set to eval mode.


In [23]:
trainer.train()

Rank 0: Instantiating SequentialSampler for single GPU.
SingleGPU Step 0 BatchOrigIndices: ['Idx:0(F:-1)'] -> FinalLoss: 13.8629


Could not estimate the number of tokens of the input, floating-point operations will not be computed


SingleGPU Step 0 BatchOrigIndices: ['Idx:1(F:-1)'] -> FinalLoss: 13.8629
SingleGPU Step 0 BatchOrigIndices: ['Idx:2(F:1)'] -> FinalLoss: 2.9828
SingleGPU Step 0 BatchOrigIndices: ['Idx:3(F:1)'] -> FinalLoss: 10.3668
SingleGPU Step 0 BatchOrigIndices: ['Idx:4(F:1)'] -> FinalLoss: 7.2344
SingleGPU Step 0 BatchOrigIndices: ['Idx:5(F:1)'] -> FinalLoss: 8.3886
SingleGPU Step 0 BatchOrigIndices: ['Idx:6(F:1)'] -> FinalLoss: 4.7914
SingleGPU Step 0 BatchOrigIndices: ['Idx:7(F:1)'] -> FinalLoss: 4.7648


Step,Training Loss
250,30.004300
500,21.486800
750,18.250400
1000,15.917100
1250,14.208600
1500,12.843200
1750,11.649000
2000,10.857600
2250,10.185300
2500,9.649900


SingleGPU Step 1 BatchOrigIndices: ['Idx:8(F:-1)'] -> FinalLoss: 13.1245
SingleGPU Step 1 BatchOrigIndices: ['Idx:9(F:-1)'] -> FinalLoss: 13.7240
SingleGPU Step 1 BatchOrigIndices: ['Idx:10(F:1)'] -> FinalLoss: 6.8639
SingleGPU Step 1 BatchOrigIndices: ['Idx:11(F:1)'] -> FinalLoss: 5.2230
SingleGPU Step 1 BatchOrigIndices: ['Idx:12(F:1)'] -> FinalLoss: 5.4882
SingleGPU Step 1 BatchOrigIndices: ['Idx:13(F:1)'] -> FinalLoss: 4.9733
SingleGPU Step 1 BatchOrigIndices: ['Idx:14(F:1)'] -> FinalLoss: 7.2032
SingleGPU Step 1 BatchOrigIndices: ['Idx:15(F:1)'] -> FinalLoss: 6.5847
SingleGPU Step 2 BatchOrigIndices: ['Idx:16(F:-1)'] -> FinalLoss: 13.7578
SingleGPU Step 2 BatchOrigIndices: ['Idx:17(F:-1)'] -> FinalLoss: 12.8266
SingleGPU Step 2 BatchOrigIndices: ['Idx:18(F:1)'] -> FinalLoss: 6.1594
SingleGPU Step 2 BatchOrigIndices: ['Idx:19(F:1)'] -> FinalLoss: 3.1979
SingleGPU Step 2 BatchOrigIndices: ['Idx:20(F:1)'] -> FinalLoss: 2.8321
SingleGPU Step 2 BatchOrigIndices: ['Idx:21(F:1)'] -> Fina

TrainOutput(global_step=3040, training_loss=14.446518657082006, metrics={'train_runtime': 13143.3661, 'train_samples_per_second': 1.85, 'train_steps_per_second': 0.231, 'total_flos': 0.0, 'train_loss': 14.446518657082006, 'epoch': 10.0})

In [17]:
model.save_pretrained(cfg.save_dir)
tokenizer.save_pretrained(cfg.save_dir)

('outputs/wpu_batch_dpo_1_7/tokenizer_config.json',
 'outputs/wpu_batch_dpo_1_7/special_tokens_map.json',
 'outputs/wpu_batch_dpo_1_7/tokenizer.json')